In [37]:
import json
import string
import numpy as np

from tqdm import tqdm
from collections import defaultdict

# Reading The Data

* base_path is the base path for all the code, datasets, and models
* the data is read using json library from python

In [9]:
base_path = "MSRVTT/raw-data/train_val_videodatainfo.json"

In [10]:
json_data = json.load(open(base_path, 'r'))

In [13]:
sentences = json_data['sentences']
videos = json_data['videos']

## Storing The Data

1. Get all the needed video id for training
2. Get all the needed captions

In [21]:
split = {'train': []}

In [22]:
count = 0

for v in videos:
    if count < 100:
        split['train'].append(int(v['id']))
        count += 1

## Get Captions

In [58]:
raw_caps_all = defaultdict(list)
raw_caps_train = defaultdict(list)
references = defaultdict(list)

In [67]:
for item in tqdm(sentences):
    if int(item['video_id'][5:]) in split['train']:
        
        vid = item['video_id']
        
        tokens = [
            token.lower() for token in item['caption'].split() if token not in string.punctuation
        ]
        
        raw_caps_all[vid].append(tokens)
        
        raw_caps_train[vid].append(tokens)
        
        references[vid].append({
            'image_id': vid, 
            'cap_id': len(references[vid]), 
            'caption': ' '.join(tokens)
        })

100%|██████████████████████████████████████████████████████████████████████| 140200/140200 [00:00<00:00, 513572.84it/s]


In [84]:
itoc = {}
split_category = {'train': defaultdict(list)}
count = 0

for item in videos:
    if count < 100:
        itoc[item['id']] = item['category']
        split_category[item['split']][int(item["category"])].append(int(item['id']))
        count += 1

In [85]:
results = {
    'split': split, 
    'raw_caps_train': raw_caps_train, 
    'raw_caps_all': raw_caps_all, 
    'references': references,
    'itoc': itoc,
    'split_category': split_category
}